# Predicción Multietiqueta con la base de datos *Planet Amazon*

Traducción al español del la [clase de Fast.ai](https://nbviewer.jupyter.org/github/fastai/course-v3/blob/master/nbs/dl1/lesson3-planet.ipynb) por [Fernando Bernuy B.](https://scholar.google.cl/citations?user=Q4tEQYYAAAAJ&hl)

## Antes de empezar...

Para evitar los problemas de la clase anterior, las bases de datos ya fueron descargadas en la máquina y solo es necesario hacer un enlace simbólico (*symlink* / *acceso directo*) de la carpeta con los datos en la ubicación correspondiente. 

Las instrucciones que nos saltaremos en este proceso quedarán comentadas para referencia futura.

In [ ]:
!mkdir ~/.fastai
!mkdir ~/.fastai/data
!mkdir ~/.fastai/data/planet
!ln -s /data/home/admin101/.fastai/data/planet/t* ~/.fastai/data/planet/

### Inicializamos el notebook

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#import torch
#torch.cuda.set_device(3)
from fastai.vision import *

## Obtener la base de datos

La base de datos *Planet Amazon* no está disponible en la [página de bases de datos de fast.ai](https://course.fast.ai/datasets) debido a restricciones de derechos de autor, pero puede ser descargado desde [Kaggle](https://www.kaggle.com). Veamos como hacerlo utilizando la [API de Kaggle](https://github.com/Kaggle/kaggle-api), ya que va a ser muy útil si quieres participar en una competencia o utilizar otra base de datos disponible en Kaggle.

Lo primero es installar la API de Kaggle descomentando y ejecutando la línea siguiente, o ejecutándola en tu terminal (dependiendo de tu plataforma puedes necesitar modificar levemente este comando para, o incluir algo similar a `source activate fastai`, o agregar una ruta específica antes del comando `pip`. Mira como `conda install` es llamado en tu plataforma la sección *Returning to work* de https://course.fast.ai/. Dependiendo de tu entorno de desarrollo, puede necesitar agregar `--user` al comando).

In [ ]:
#! {sys.executable} -m pip install kaggle --upgrade

Luego debes subir tus credenciales de Kaggle en tu instancia. Ingresa a tu cuenta de [Kaggle](https://www.kaggle.com), luego entra a la página de tu perfil, y en ella a la sección "My Account". Aquí, busca el botón  llamado "*Create New API Token*" y haz click en él. Esto iniciará la descarga de un archivo llamado "kaggle.json".

Sube este archivo al directorio en el que se está ejecutando este *notebook* haciendo click en "Upload" en la página principal de tu *Jupyter* (o utilizando los comandos para Colab a continuación) y luego ejecuta los comandos de Linux o Windows según corresponda (si estás utilizando Colab, entonces usa los comandos de Linux).

In [ ]:
# ---- Para subir archivo en Colab ----
#from google.colab import files
#uploaded = files.upload()

In [ ]:
# ---- Para mover en Linux or Colab ----
#! mkdir -p ~/.kaggle/
#! mv kaggle.json ~/.kaggle/

# ---- Para mover en Windows ----
# ! mkdir %userprofile%\.kaggle
# ! move kaggle.json %userprofile%\.kaggle

Ya está todo configurado para descargar los datos de la [competencia planet understanding](https://www.kaggle.com/c/planet-understanding-the-amazon-from-space). **Primero debes ir a su página principal y aceptar sus reglas**, y luego ejecutar las dos celdas siguientes (descomenta los comandos para descargar y descomprimir la data). Si recibes un error `403 forbidden` significa que no has aceptado las reglas de la competencia aún, y debes ir a la página de la competencia, hacer click en el tab *Rules* y hacer click en el boton *accept*.

In [ ]:
path = Config.data_path()/'planet'
path.mkdir(parents=True, exist_ok=True)
path

In [ ]:
#! kaggle competitions download -c planet-understanding-the-amazon-from-space -f train-jpg.tar.7z -p {path}  
#! kaggle competitions download -c planet-understanding-the-amazon-from-space -f train_v2.csv -p {path}  
#! unzip -q -n {path}/train_v2.csv.zip -d {path}

Para extraer el contenido de este archivo, vamos a necesitar *7zip*, asi que descomenta la siguiente line si necesitas instalarlo. (o ejecuta `sudo apt install p7zip-full`).

In [ ]:
# ! conda install --yes --prefix {sys.prefix} -c haasad eidl7zip

Y ahora podemos descomprimir la base de datos (descomenta para hacerlo, puede tomar algunos minutos)

In [ ]:
#! 7za -bd -y -so x {path}/train-jpg.tar.7z | tar xf - -C {path.as_posix()}

# Clasificación Multietiqueta

Contratrio a la base de datos de mascotas estudiada en la lección anterior, aquí cada imagen puede tener múltiples etiquetas. Si miramos el archivo *csv* que contiene las etiquetas (en `train_v2.csv`) podemos ver que cada `image_name` está asociada a varias etiquetas separadas por espacios:

In [ ]:
df = pd.read_csv(path/'train_v2.csv')
df.head()

Para incluir esto en un `DataBunch` usando la [API de data block](https://docs.fast.ai/data_block.html) necesitamos usar la clase `ImageList` ( y no la clase `ImageDataBunch`). Esto se asegurará que el modelo creado tendrá la función de pérdida apropiada para manejar múltiples etiquetas.

Utilizamos paréntesis entorno al *pipeline* (secuencia de procesos) de *data block* para que podamos hacer una declaración de múltiples líneas sin tener que agregar `\` al final de cada una.

In [ ]:
np.random.seed(42)
src = (ImageList.from_csv(path,
                          'train_v2.csv',
                          folder='train-jpg',
                          suffix='.jpg')
       .split_by_rand_pct(0.2)
       .label_from_df(label_delim=' '))

In [ ]:
# src = ImageList.from_csv(path,'train_v2.csv',folder='train-jpg',suffix='.jpg').split_by_rand_pct(0.2).label_from_df(label_delim=' ')

In [ ]:
# src = ImageList.from_csv(path,'train_v2.csv',folder='train-jpg',suffix='.jpg')
# src = src.split_by_rand_pct(0.2)
# src = src.label_from_df(label_delim=' ')

Finalmente agregamos las transformaciones y generamos el databunch.  (ver [get_transforms](https://docs.fast.ai/vision.transform.html#get_transforms) )

In [ ]:
tfms = get_transforms(flip_vert=True,
                      max_lighting=0.1,
                      max_zoom=1.05,
                      max_warp=0.)

In [ ]:
data = (src.transform(tfms, size=128)
        .databunch().normalize(imagenet_stats))

`show_batch` también funciona en este caso y nos muestra las diferentes etiquetas saparadas por `;`

In [ ]:
data.show_batch(rows=3, figsize=(12,9))

Para crear un `Learner` utilizamos la misma función de la lección 1, y la arquitectura base también es `resnet50`, pero las métricas son un poco diferentes: utilizamos [`accuracy_thresh`](https://docs.fast.ai/metrics.html#accuracy_thresh) en vez de `accuracy`. En la lección anterior determinamos la predicción de la clase de una imagen al escoger la activación final con el valor más alto, pero aquí cada activación puede estar entre 0 y 1. `accuract_thresh` elige las clases cuya activación sea mayor que un cierto umbral (0.5 por defecto) y las compara con el *ground thruth*.

Finalmente, la competencia de Kaggle utiliza la métrica Fbeta (más detalles [aqui](https://en.wikipedia.org/wiki/F1_score))

In [ ]:
arch = models.resnet50

In [ ]:
acc_02 = partial(accuracy_thresh, thresh=0.2)
f_score = partial(fbeta, thresh=0.2)
learn = cnn_learner(data, arch, metrics=[acc_02, f_score])

Utilizamos el método `LR Finder` para encontrar el mejor *learning rate*. [¿Por qué?¿Cómo?](https://www.jeremyjordan.me/nn-learning-rate/)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

Elegimos un *learning rate* apropiado de acuerdo al grafico

In [ ]:
lr = 0.01

In [ ]:
lr = 0.01
learn.fit_one_cycle(5, slice(lr))

In [ ]:
learn.save('stage-1-rn50')

In [ ]:
learn.recorder.plot_losses()

... y ahora hacemos *fine-tuning* del modelo completo

In [ ]:
learn.unfreeze()

Buscamos el LR adecuado para esta nueva etapa

In [ ]:
learn.lr_find()
learn.recorder.plot()

nuevamente elegimos un *learning rate* apropiado y volvemos a entrenar

In [ ]:
learn.fit_one_cycle(5, slice(2e-5, lr/5))

In [ ]:
learn.save('stage-2-rn50')

In [ ]:
# learn.load('stage-2-rn50')

### Cambio de Tamaño

Un posible método para seguir mejorando el rendimiento de la red es utilizar un tamaño mayor de la imagen de entrada. Para eso cambiaremos la forma del databunch y repetiremos el proceso de entrenamiento: primero las últimas capas y luego la red completa utilizando `lr_find`

In [ ]:
data = (src.transform(tfms, size=256)
        .databunch().normalize(imagenet_stats))
learn.data = data
data.train_ds[0][0].shape

In [ ]:
learn.freeze()

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
lr=1e-2/2

In [ ]:
learn.fit_one_cycle(5, slice(lr))

In [ ]:
learn.save('stage-1-256-rn50')

In [ ]:
learn.recorder.plot_losses()
learn.recorder.plot_lr()

## Y ahora continuamos con la red completa

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(5, slice(1e-5, lr/5))

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.save('stage-2-256-rn50')

En la práctica, no podrás saber qué tan bien vas hasta que lo hayas enviado a Kaggle, ya que los resultados se miden con una base e datos distinta a la que tenemos para entrenamiento. A modo de referencia, el lugar 50 (de 938 equipos) en el ranking privado tuvo un puntaje de `0.930`. ([ver leaderboard](https://www.kaggle.com/c/planet-understanding-the-amazon-from-space/leaderboard))

In [ ]:
learn.export()

In [ ]:
learn.destroy()

In [ ]:
! rm -r .fastai/data

# FIN
